In [84]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
!pip install transformers
from transformers import BertTokenizer, BertForSequenceClassification

# Load the dataset
df = pd.read_csv('/content/Laptop_datasets.csv')  # Replace 'dataset.csv' with the path to your dataset file

# Preprocessing the dataset
text_data = df['text'].values
aspect_data = df['aspect'].values
label_data = df['label'].values

# Map labels to numerical values
label_mapping = {'positive': 2, 'neutral': 1, 'negative': 0}
label_data = [label_mapping[label] for label in label_data]

# Split the dataset into training and testing sets
text_train, text_test, aspect_train, aspect_test, label_train, label_test = train_test_split(
    text_data, aspect_data, label_data, test_size=0.2, random_state=42
)




Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [85]:
text_train[12]

"With a mac you don't have to worry about antivirus software or firewall, it's so wonderful."

In [86]:
text_test[0]

'It is easy to use, has great screen quality, and every so light weight.'

In [87]:
label_train[0:10]

[0, 0, 0, 1, 0, 0, 0, 0, 2, 2]

In [88]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


In [89]:
tokenizer

BertTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [90]:
# Tokenize the input data

input_ids_train = []
attention_masks_train = []
for text, aspect in zip(text_train, aspect_train):
    encoded = tokenizer.encode_plus(
        text,
        aspect,
        add_special_tokens=True,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )

In [91]:
input_ids_train.append(encoded['input_ids'].squeeze())
attention_masks_train.append(encoded['attention_mask'].squeeze())


In [92]:
encoded,

({'input_ids': tensor([[  101,  1996,  2640,  2003, 12476,  1010,  3737,  2003, 15741,  1012,
            102,  3737,   102,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,   

In [93]:
len(encoded)

3

In [94]:
type(encoded)

transformers.tokenization_utils_base.BatchEncoding

In [95]:
input_ids_train[0],type(input_ids_train)

(tensor([  101,  1996,  2640,  2003, 12476,  1010,  3737,  2003, 15741,  1012,
           102,  3737,   102,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

In [96]:
len(input_ids_train)

1

In [97]:
attention_masks_train[0],len(attention_masks_train),type(attention_masks_train)

(tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0

In [98]:

   
input_ids_test = []
attention_masks_test = []
for text, aspect in zip(text_test, aspect_test):
    encoded = tokenizer.encode_plus(
        text,
        aspect,
        add_special_tokens=True,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    input_ids_test.append(encoded['input_ids'].squeeze())
    attention_masks_test.append(encoded['attention_mask'].squeeze())

input_ids_train = torch.stack(input_ids_train)
attention_masks_train = torch.stack(attention_masks_train)
label_train = torch.tensor(label_train)

In [99]:
text_train[0]

'You have to toss out the wifi card and replace it just to have any sort of network capability.'

In [100]:
input_ids_train,input_ids_train.shape

(tensor([[  101,  1996,  2640,  2003, 12476,  1010,  3737,  2003, 15741,  1012,
            102,  3737,   102,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,   

In [101]:
type(input_ids_train)

torch.Tensor

In [102]:
attention_masks_train[0],len(attention_masks_train)

(tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0

In [103]:
type(attention_masks_train)

torch.Tensor

In [104]:
attention_masks_train[0].shape

torch.Size([512])

In [105]:
input_ids_test = torch.stack(input_ids_test)
attention_masks_test = torch.stack(attention_masks_test)
label_test = torch.tensor(label_test)

In [106]:
text_test[0],len(text_test[0])

('It is easy to use, has great screen quality, and every so light weight.', 71)

In [107]:
input_ids_test[0],input_ids_test.shape

(tensor([ 101, 2009, 2003, 3733, 2000, 2224, 1010, 2038, 2307, 3898, 3737, 1010,
         1998, 2296, 2061, 2422, 3635, 1012,  102, 2224,  102,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,  

In [108]:
len(input_ids_test),len(input_ids_test[0])

(463, 512)

In [109]:
label_test[4],type(label_test[0])

(tensor(0), torch.Tensor)

In [111]:
# Create DataLoader
train_dataset = TensorDataset(input_ids_train, attention_masks_train, label_train)
test_dataset = TensorDataset(input_ids_test, attention_masks_test, label_test)

batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

# Define the LSTM+CNN model
class LSTM_CNN(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, num_classes):
        super(LSTM_CNN, self).__init__()
        self.bert = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_classes)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.conv1d = nn.Conv1d(hidden_dim, 128, kernel_size=3, padding=1)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(128, num_classes)

    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.bert(input_ids=input_ids, attention_mask=attention_mask, return_dict=False)
        lstm_output, _ = self.lstm(pooled_output.unsqueeze(1))
        lstm_output = lstm_output.permute(0, 2, 1)  # Reshape for conv1d
        conv_output = self.conv1d(lstm_output)
        conv_output = self.dropout(conv_output)
        conv_output = torch.max(conv_output, dim=2)[0]  # Global max pooling
        logits = self.fc(conv_output)
        return logits

AssertionError: ignored